In [0]:
!pip install tensorflow-gpu==2.0.0

## New Ways Building Network

In [0]:
import tensorflow as tf

In [0]:
print("Tensorflow Version: {}".format(tf.__version__))

Tensorflow Version: 2.0.0


## Eager Execution

Eager execution is one of the biggest changes between Tensorflow 1.x and 2.x. 

### Basic Concept

In Tensorflow 1, all operations on the tensor are under an active session. A simple example is below.

```python
# Tensorflow 1.x
with tf.Session() as sess:
  add_res = tf.constant(2.0, dtype=float) + tf.constant(1.5, dtype=float)
  print(sess.run(add_res))   # 3.5
```

In Tensorflow 2 with **eager** mode, you no need start a session to run the operation, rather you can do calculate more like in native python.

In [0]:
# tensor
tf.constant(2.0, dtype=float) + tf.constant(1.5, dtype=float)

<tf.Tensor: id=2, shape=(), dtype=float32, numpy=3.5>

You can even get the calculation result by simply calling a method `numpy()`.

In [0]:
# numpy result (default float type is tf.float32)
(tf.constant(2.0, dtype=float) + tf.constant(1.5, dtype=float)).numpy()

3.5

In Tensorflow 2.0, eager execution is on by default. Unlike Tensorflow 1.x, operations under the eager model are executing immediately without building a graph.

### No Initializer Required

In Tensorflow 1.x, a simple variable must be initialized after it was declared.

In Tensorflow 2.x, no matter in `eager` execution or in `tf.function()` no initialization operation is required.

An example in the **eager** mode. 

In [0]:
v_add_seq = tf.Variable(1.0)

for i in range(10):
  # this step is not adding an operation to the graph
  # you can simply think this is equivalent to
  # `v_add_seq += i`
  v_add_seq.assign_add(i)
  print(v_add_seq)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=4.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=7.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=11.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=16.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=22.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=29.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=37.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=46.0>


The result of the below script is equal to the above one but different from the types of them. One is a variable and the other is a constant.

In [0]:
v_add_seq = tf.Variable(1.0)

for i in range(10):
  v_add_seq = v_add_seq + i
  print(v_add_seq)

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)
tf.Tensor(11.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)
tf.Tensor(22.0, shape=(), dtype=float32)
tf.Tensor(29.0, shape=(), dtype=float32)
tf.Tensor(37.0, shape=(), dtype=float32)
tf.Tensor(46.0, shape=(), dtype=float32)


In Tensorflow 1.x, the above script would create a sequential of operations for adding values.

```python
# Tensorflow 1.x
tf.reset_default_graph()
with tf.Graph().as_default():
  v_add_seq = tf.Variable(1.0)
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5):
      # a `wrong` way to add an integer
      # the following script would add operations on the graph
      v_add_seq = tf.assign_add(v_add_seq, i)
      """
      +0 = 1.0
      +1 = 2.0
      +2 = 5.0
      +3 = 11.0
      +4 = 21.0
      """
      print("+{} = {}".format(i, sess.run(v_add_seq)))

      # a correct way to add the integer sequentially
      new_value = tf.assign_add(v_add_seq, i)
      print("+{} = {}".format(i, sess.run(v_add_seq)))
```

Example In the `tf.function()`.

In [0]:
v_add_seq_2 = tf.Variable(1.0)

@tf.function()
def add_seq(x):
  return v_add_seq_2.assign_add(x)

for i in range(10):
  print(add_seq(i))

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)
tf.Tensor(11.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)
tf.Tensor(22.0, shape=(), dtype=float32)
tf.Tensor(29.0, shape=(), dtype=float32)
tf.Tensor(37.0, shape=(), dtype=float32)
tf.Tensor(46.0, shape=(), dtype=float32)


## TF Function

### Basic Concept

`tf.function()` is a new decorator declaring operations in Tensorflow 2.0.

The basic declearation is like below:
```python
# the necessary decorator
@tf.function
def func(*params):
  '''define the operations'''
  results = ...
  return results
```

With `tf.function()`, you **don't** need the following:
* running via `tf.Session()`
* using `tf.control_dependencies()`
* using `tf.global_variables_initializers()`
* using `tf.cond()` and `tf.while_loop()`.

With `tf.function()`, you are supported to **do** the following:
* building concrete functions

In [0]:
@tf.function
def add(a, b):
  '''An add operation.'''
  return a + b

You can use `tf.function()` to process tensor. The `tf.function()` can return the result as the tensor.

In [0]:
add(tf.constant(2.0, dtype=float), tf.constant(3.5, dtype=float))

<tf.Tensor: id=593, shape=(), dtype=float32, numpy=5.5>

In [0]:
add(tf.constant(3.0, dtype=float), tf.constant(7.5, dtype=float)).numpy()

10.5

* You can also use `tf.function()` as the normal python function.

In [0]:
add(1.1, 1.3)

<tf.Tensor: id=601, shape=(), dtype=float32, numpy=2.4>

Furthermore, you might not need to care about the data type of tensors while doing the same operation. 

Compare to Tensorflow 1.x, you don't need to declare the same operation but for different data types, for example, for integer or for string variables.

In [0]:
add("a", "a")

<tf.Tensor: id=128, shape=(), dtype=string, numpy=b'aa'>

### More Faster

One of the biggest changes between Tensorflow 1.x and 2.x is the eager mode. It is no required to execute the operations on a graph under a session via `tf.Session()`.

But with `tf.function()`, the calculation would be faster then eager execution. (Due to the optimization in `tf.function()`.)

Here we demo the time consumption between **eager model** and **tf.function()**. 

LSTM cell is one type of gated RNN cell improving memorizing mechanism along with time-series data. But the state in each time moment depended on the previous time moment. This dependent calculation causes itself to consume more time and hard to accelerate during the training.

In [0]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def fn(input, state):
  return lstm_cell(input, state)

In [0]:
input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2

import timeit
print("Eager mode: {}".format(timeit.timeit(lambda: lstm_cell(input, state), number=10)))
print("tf.function mode: {}".format(timeit.timeit(lambda: fn(input, state), number=10)))

Eager mode: 0.641535319000468
tf.function mode: 0.06657306199940649


### Automatic Control Dependencies

In Tensorflow 1.x, we use `tf.control_dependencies()` to control the partial calculation order in the graph. But it is easy to ignore the updated relation among tensors.

A simple example is below.

```python
# Tensorflow 1.x

with tf.Graph().as_default():
  v1 = tf.Variable(tf.constant(1.0))
  v2 = tf.Variable(2.0)
  x = tf.placeholder(tf.float32)
  y = tf.placeholder(tf.float32)
  
  # =============================================
  # wrong way (easier way to build the operation)
  # =============================================
  #v1_assign = v1.assign(y * v2)
  #v2_assign = v2.assign_add(x * v1)
  #v_sum = v1 + v2  # 3.0
  
  # ===============================================================
  # correct way (have to notice the dependencies among several operations)
  # ===============================================================
  v1_assign = v1.assign(y * v2)
  with tf.control_dependencies([v1_assign]):
    # control is dependent on operator v1_assign
    v2_assign = v2.assign_add(x * v1)
    with tf.control_dependencies([v2_assign]):
      # control is dependent on both operators v1_assign, v2_assign
      v_sum = v1 + v2
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([v1, v2, v_sum], feed_dict={x: 3.0, y: 4.0}))  # [8.0, 26.0, 34.0]
```

In Tensorflow 1.x, `tf.control_dependency()` controls the calculation order in the flow, especially for the operations dependent on the previous and next results. Lots of loop-like operations would encounter such problems. (It is really not convenient to building partial operation.)

In Tensorflow 2.x, the control dependencies are handed over to Tensorflow, you can simply build the operation like native python.

In [0]:
v1 = tf.Variable(1.0)
v2 = tf.Variable(2.0)

@tf.function
def fn_assign(x, y):
  v1.assign(y * v2)       # 8
  v2.assign_add(x * v1)   # 26 = 2 + 24
  return v1 + v2          # 34 = 8 + 26

print("{}\n{}\n{}".format(v1, v2, fn_assign(3.0, 4.0)))

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=8.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=26.0>
34.0


### Polymorphic / Concrete Function

Tensorflow 2.0 with eager mode makes development and debugging more interactive, but it is not allowed for deploying on devices. More operations are required while exporting a saved model or deploying on devices. It requires a **concrete function** defining the graph that can be exported into a saved model. A concrete function can be exported from a polymorphic function. A polymorphic function means python callable that encapsulates several concrete function graphs behind one API. (You can simply think it is required to define tensor spec as passed parameters to the function.)

There are three basic steps to define or export a concrete function.
* Define `input_signature` parameter in `tf.function`.
* Pass in tf.TenorSpec into `get_concrete_function`, e.g. tf.TensorSpec(shape=[1], dtype=tf.float32).
* Pass a sample input tensor into `get_concrete_function`, e.g. tf.constant(2.0, shape=[1]).

In [0]:
@tf.function
def simple_add(v1, v2):
  """Add two values, v1 and v2, with their relative data type."""
  return v1 + v2

In [0]:
c = simple_add.get_concrete_function(tf.TensorSpec(shape=None, dtype=float),
                                     tf.TensorSpec(shape=None, dtype=float))

print(c(tf.constant(2.0), tf.constant(3.5)))

tf.Tensor(5.5, shape=(), dtype=float32)


You can use attributes `graph` accessing the graph and its relative operations.

In [0]:
print(c.graph)
print(c.graph.inputs)
print(c.graph.outputs)

FuncGraph(name=simple_add, id=140623849765576)
[<tf.Tensor 'v1:0' shape=<unknown> dtype=float32>, <tf.Tensor 'v2:0' shape=<unknown> dtype=float32>]
[<tf.Tensor 'Identity:0' shape=<unknown> dtype=float32>]


### Tensors like Objects

In [0]:
state = []

@tf.function
def append_list(x):
  if len(state) < 1:
    state.append(tf.Variable(2.0))
  return state[0] * x

print(append_list(tf.constant(1.0)))
print(append_list(tf.constant(2.0)))
print(state)

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>]


### Easier Control Flow

No more need `tf.cond()` and `tf.while_loop()`. 

In Tensorflow 2.x, you can operate the tensor in the same way as usual in python while for branch control.

In [0]:
@tf.function
def while_loop(x):
  while tf.reduce_sum(x) > 1:
    x = tf.tanh(x)
  return x

while_loop(tf.random.uniform([10]))

<tf.Tensor: id=48, shape=(10,), dtype=float32, numpy=
array([0.10283414, 0.10629437, 0.10663301, 0.1084716 , 0.10902831,
       0.10768783, 0.09392301, 0.10888813, 0.10447586, 0.05031879],
      dtype=float32)>

It is easy to use `for loop` to operate the tensor. 

In [0]:
@tf.function
def for_loop(x):
  # notice index (i) here is a Tensor, its type is placeholder
  # so that you can't replace `tf.range` with `range`
  for i in tf.range(10):
    x += i
  return x

print(for_loop(tf.Variable(0)))

tf.Tensor(45, shape=(), dtype=int32)


Easier way operates a TensorArray data structure.

In [0]:
@tf.function
def write_array(x):
  ta = tf.TensorArray(tf.float32, size=10)
  for i in tf.range(10):
    x += tf.cast(i, float)
    ta = ta.write(i, x)
  return ta.stack()

write_array(1.0).numpy()

array([ 1.,  2.,  4.,  7., 11., 16., 22., 29., 37., 46.], dtype=float32)